In [3]:
import numpy as np
import tensorflow as tf
from tensorflow.keras.preprocessing.text import Tokenizer
from tensorflow.keras.preprocessing.sequence import pad_sequences
from tensorflow.keras.models import Sequential
from tensorflow.keras.layers import Bidirectional, Dense Embedding, LSTM
import requests

#download Shakespeare text
def download_text():
    url = "https://www.gutenberg.org/files/100/100-0.txt"
    response = requests.get(url)
    text = response.text
    start_index = text.find("THE SONNETS")
    end_index = text.find("THE END")
    return text[start_index:end_index]

dataset_text = download_text()

#tokenization
tokenizer = Tokenizer()
tokenizer.fit_on_texts([dataset_text])
total_words = len(tokenizer.word_index) + 1

#convert text to sequences
input_sequences = []
for line in dataset_text.split('\n'):
    token_list = tokenizer.texts_to_sequences([line])[0]
    for i in range(1, len(token_list)):
        input_sequences.append(token_list[:i+1])

max_sequence_length = max(len(seq) for seq in input_sequences)
input_sequences = pad_sequences(input_sequences, maxlen=max_sequence_length, padding='pre')

X, y = input_sequences[:, :-1], input_sequences[:, -1]
y = tf.keras.utils.to_categorical(y, num_classes=total_words)



In [4]:
model = Sequential([
    Embedding(total_words, 64),
    Bidirectional(LSTM(150, activation='tanh', return_sequences=True)),
    Bidirectional(LSTM(150, activation='tanh', return_sequences=True)),
    Bidirectional(LSTM(150, activation='tanh')),
    Dense(total_words, activation='softmax')
])

model.compile(loss='categorical_crossentropy', optimizer='adam', metrics=['accuracy'])


In [5]:
model.fit(X, y, epochs=100, verbose=1)


Epoch 1/100
550/550 ━━━━━━━━━━━━━━━━━━━━ 50s 73ms/step - accuracy: 0.1155 - loss: 6.6417
Epoch 2/100
550/550 ━━━━━━━━━━━━━━━━━━━━ 41s 74ms/step - accuracy: 0.1242 - loss: 5.9643
Epoch 3/100
550/550 ━━━━━━━━━━━━━━━━━━━━ 41s 75ms/step - accuracy: 0.1281 - loss: 5.8442
Epoch 4/100
550/550 ━━━━━━━━━━━━━━━━━━━━ 42s 76ms/step - accuracy: 0.1306 - loss: 5.7580
Epoch 5/100
550/550 ━━━━━━━━━━━━━━━━━━━━ 41s 74ms/step - accuracy: 0.1336 - loss: 5.6842
Epoch 6/100
550/550 ━━━━━━━━━━━━━━━━━━━━ 41s 75ms/step - accuracy: 0.1346 - loss: 5.6343
Epoch 7/100
550/550 ━━━━━━━━━━━━━━━━━━━━ 41s 75ms/step - accuracy: 0.1347 - loss: 5.5749
Epoch 8/100
550/550 ━━━━━━━━━━━━━━━━━━━━ 41s 74ms/step - accuracy: 0.1291 - loss: 5.5943
Epoch 9/100
550/550 ━━━━━━━━━━━━━━━━━━━━ 41s 74ms/step - accuracy: 0.1355 - loss: 5.4864
Epoch 10/100
550/550 ━━━━━━━━━━━━━━━━━━━━ 41s 74ms/step - accuracy: 0.1410 - loss: 5.4005
Epoch 11/100
550/550 ━━━━━━━━━━━━━━━━━━━━ 42s 75ms/step - accuracy: 0.1380 - loss: 5.3297
Epoch 12/100
550/55

In [6]:
def generate_text(seed_text, next_words=10):
    for _ in range(next_words):
        token_list = tokenizer.texts_to_sequences([seed_text])[0]
        token_list = pad_sequences([token_list], maxlen=max_sequence_length-1, padding='pre')
        predicted = np.argmax(model.predict(token_list), axis=-1)
        
        output_word = next((word for word, index in tokenizer.word_index.items() if index == predicted), "")
        
        if output_word:
            seed_text = seed_text.strip() + " " + output_word
            
    return seed_text

print(generate_text("Blue sky"))

1/1 ━━━━━━━━━━━━━━━━━━━━ 1s 899ms/step
1/1 ━━━━━━━━━━━━━━━━━━━━ 0s 28ms/step
1/1 ━━━━━━━━━━━━━━━━━━━━ 0s 28ms/step
1/1 ━━━━━━━━━━━━━━━━━━━━ 0s 27ms/step
1/1 ━━━━━━━━━━━━━━━━━━━━ 0s 28ms/step
1/1 ━━━━━━━━━━━━━━━━━━━━ 0s 28ms/step
1/1 ━━━━━━━━━━━━━━━━━━━━ 0s 28ms/step
1/1 ━━━━━━━━━━━━━━━━━━━━ 0s 27ms/step
1/1 ━━━━━━━━━━━━━━━━━━━━ 0s 28ms/step
1/1 ━━━━━━━━━━━━━━━━━━━━ 0s 28ms/step
Blue sky is my verse so barren of new pride 


In [7]:
print(generate_text("Roses are red"))

1/1 ━━━━━━━━━━━━━━━━━━━━ 0s 28ms/step
1/1 ━━━━━━━━━━━━━━━━━━━━ 0s 29ms/step
1/1 ━━━━━━━━━━━━━━━━━━━━ 0s 28ms/step
1/1 ━━━━━━━━━━━━━━━━━━━━ 0s 27ms/step
1/1 ━━━━━━━━━━━━━━━━━━━━ 0s 26ms/step
1/1 ━━━━━━━━━━━━━━━━━━━━ 0s 27ms/step
1/1 ━━━━━━━━━━━━━━━━━━━━ 0s 28ms/step
1/1 ━━━━━━━━━━━━━━━━━━━━ 0s 28ms/step
1/1 ━━━━━━━━━━━━━━━━━━━━ 0s 27ms/step
1/1 ━━━━━━━━━━━━━━━━━━━━ 0s 27ms/step
Roses are red which i into my verse 


In [8]:
print(generate_text("Paint it black"))

1/1 ━━━━━━━━━━━━━━━━━━━━ 0s 27ms/step
1/1 ━━━━━━━━━━━━━━━━━━━━ 0s 27ms/step
1/1 ━━━━━━━━━━━━━━━━━━━━ 0s 26ms/step
1/1 ━━━━━━━━━━━━━━━━━━━━ 0s 27ms/step
1/1 ━━━━━━━━━━━━━━━━━━━━ 0s 27ms/step
1/1 ━━━━━━━━━━━━━━━━━━━━ 0s 28ms/step
1/1 ━━━━━━━━━━━━━━━━━━━━ 0s 28ms/step
1/1 ━━━━━━━━━━━━━━━━━━━━ 0s 28ms/step
1/1 ━━━━━━━━━━━━━━━━━━━━ 0s 29ms/step
1/1 ━━━━━━━━━━━━━━━━━━━━ 0s 27ms/step
Paint it black a seething truth doth stol’n from thee 


In [9]:
print(generate_text("Love is"))

1/1 ━━━━━━━━━━━━━━━━━━━━ 0s 27ms/step
1/1 ━━━━━━━━━━━━━━━━━━━━ 0s 26ms/step
1/1 ━━━━━━━━━━━━━━━━━━━━ 0s 28ms/step
1/1 ━━━━━━━━━━━━━━━━━━━━ 0s 26ms/step
1/1 ━━━━━━━━━━━━━━━━━━━━ 0s 27ms/step
1/1 ━━━━━━━━━━━━━━━━━━━━ 0s 27ms/step
1/1 ━━━━━━━━━━━━━━━━━━━━ 0s 27ms/step
1/1 ━━━━━━━━━━━━━━━━━━━━ 0s 29ms/step
1/1 ━━━━━━━━━━━━━━━━━━━━ 0s 28ms/step
1/1 ━━━━━━━━━━━━━━━━━━━━ 0s 29ms/step
Love is too young to know what conscience is 


In [10]:
print(generate_text("Blood"))

1/1 ━━━━━━━━━━━━━━━━━━━━ 0s 27ms/step
1/1 ━━━━━━━━━━━━━━━━━━━━ 0s 27ms/step
1/1 ━━━━━━━━━━━━━━━━━━━━ 0s 27ms/step
1/1 ━━━━━━━━━━━━━━━━━━━━ 0s 28ms/step
1/1 ━━━━━━━━━━━━━━━━━━━━ 0s 28ms/step
1/1 ━━━━━━━━━━━━━━━━━━━━ 0s 27ms/step
1/1 ━━━━━━━━━━━━━━━━━━━━ 0s 27ms/step
1/1 ━━━━━━━━━━━━━━━━━━━━ 0s 26ms/step
1/1 ━━━━━━━━━━━━━━━━━━━━ 0s 28ms/step
1/1 ━━━━━━━━━━━━━━━━━━━━ 0s 28ms/step
Blood didst i still and yet not your defect 
